<a href="https://colab.research.google.com/github/Nomade25/Mackenzie/blob/main/DesafioTitanic_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mlp
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings ("ignore")

 
import matplotlib.pyplot as graph
%matplotlib inline
graph.rcParams['figure.figsize'] = (15,5)
graph.rcParams["font.family"] = 'DejaVu Sans'
graph.rcParams["font.size"] = '12'
graph.rcParams['image.cmap'] = 'rainbow'
 
 
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
dataset_titanic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Arquivos/train_titanic.csv") 

In [ ]:
dataset_titanic.head()
# Temos:
# Id do Passageiro
# SObreviventes (varivavel dependente)
# Pclass (classe do passageiro)
# Name, Sex, Age
# SibSP  relação familiar: sibsp (sobrinho, sobrinha, irmãos e meios irmão) | Spouse (marido e esposa)
# Parch relação familiar: parent(mãe e pai) | Child (filho ou filha e enteados)
# Ticket (nº do ticket)
# fare tarifa do passageiro
# Cabin nº da cabine
# Embarked porto que o passageiro embarcou




In [ ]:
# Olhando o dataset completo

dataset_titanic.hist(bins=50, figsize=(10,10))
plt.show()

In [ ]:
# quais as colunas com valores vazios ou nulos?

(dataset_titanic.isnull().sum()).sort_values(ascending=False)

In [ ]:
# Verificando tipos de atributo
# Temos varios dados do tipo float que podemos normatizar

dataset_titanic.dtypes

In [ ]:
# Agora vou pegar a idade (age) e substituir nulo pela média
dataset_titanic01 = dataset_titanic.fillna(dataset_titanic['Age'].mean())

dataset_titanic01.head()

# Podemos percerber abaixo que para esse novo dataframe idade agora está vazio

(dataset_titanic01.isnull().sum()).sort_values(ascending=False)


Embarked       0
Cabin          0
Fare           0
Ticket         0
Parch          0
SibSp          0
Age            0
Sex            0
Name           0
Pclass         0
Survived       0
PassengerId    0
dtype: int64

In [ ]:
dataset_titanic01.shape

In [ ]:
# podemos verificar o resumo desses dados
dataset_titanic01.describe()

# Todos atributos contem 891 instâncias
# a média de idade de passageiros  sobreviventes é 29
# PClass são dividos por Alta, Classe média e baixa, mostra 35% estão no primeiro quartil
# SibSp define a relação de class entre familiares sobreviventes (irmãos, irmãs, meios, marido e esposas)
# Parch define a relação de classe entre familiares sobreviventes entre (pai, mãe, filho, filha e etc...)
# sexo dos passageiros sobreviventes
# fare valor que o passgeiro pagou

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# Existe alguma correlação dos atributos com sobreviventes? vamos pesquisar...

corr = dataset_titanic01.corr () 
maior_corr = corr.index[abs(corr["Survived"])>0.1]
maior_corr   

# conseguimos encontra somente correlação fraca <0.10

In [ ]:
# Plotar o gra´fico de calor dessas variaveis 
# por enquanto vamos considerar classe dos passageiros e valor médio da tarifa
plt.figure(figsize=(8,8))
ax = sns.heatmap(dataset_titanic01[maior_corr].corr(),annot=True, cmap="Blues")
 

In [ ]:
# vamos considerar o seguinte atributo pra regressão logistica 
# O quanto a classe o passageiro explica a sobrevivencia do mesmo

train_y = dataset_titanic01["Survived"]

train_x = dataset_titanic01["Pclass"]

graph.scatter(train_x, train_y, c = train_y, marker = 'D')

graph.yticks([0, 1], ['No', 'Yes'])
graph.ylabel("Survived")
graph.ylim([-0.5, 1.5])
graph.xlabel("Pclass")

graph.show()
# conforme podemos ver abaixo no modelo de treino os sobreviventes estão espalhado por todas as classe,
# classe não é determinante para sobreviver, mas pode ser que estatisticamente existe uma maior frequencia


In [ ]:
# Podemos plotar os sobrevivente por clase pra constatar a distribuição

sns.barplot(x="Pclass", y= "Survived", data= dataset_titanic01)

Conforme podemos ver acima, a conclusão inicial que teremos que trabalhar mais as features, para conseguir chegar numa boa regressão, vamos seguir com as trativas abaixo, pois pegando apenas o dataset direto e fazendo algumas tratativas não foi possivel chegar num resultado satisfatório.

In [ ]:
# Como esta a questão dos nulos
(dataset_titanic.isnull().sum()).sort_values(ascending=False)


In [ ]:
# vamos montar uma categorias de idade  a partir da feature Age
# e criar assim sobreviventes por faixa etária

bins= [-1,0,5,12,18,24,35,60, np.inf]
labels= ['Unknow','Baby', 'Child', 'Teenager', 'Student', 'Young Adult','Adult', 'Senior']
dataset_titanic['AgeGroup'] = pd.cut(dataset_titanic["Age"], bins, labels =labels)
sns.barplot(x="AgeGroup",y="Survived", data= dataset_titanic)
plt.show()


In [ ]:
# Como cabine tem poucos preenchimentos tranformando em bit 0 ou 1

dataset_titanic["CabinBool"]= (dataset_titanic["Cabin"].notnull().astype('int'))

sns.barplot(x='CabinBool', y= 'Survived', data= dataset_titanic)

In [ ]:
# Como estão os nulos
(dataset_titanic.isnull().sum()).sort_values(ascending=False)

In [ ]:
 #Limpar dados irrelevantes agora, pois já extraimos os dados de cabine que passou 
 # a assumir CabinBool
dataset_titanic = dataset_titanic.drop(['Cabin','Ticket'],axis=1)


In [ ]:
# Como está a questão dos nulos agora
(dataset_titanic.isnull().sum()).sort_values(ascending=False)

In [ ]:
# Vmos pegar esses 2 passegeiros que o embarque esta nulo e preencher com algum valor mais constante
dataset_titanic = dataset_titanic.fillna({"Embarked": "S"})

# Agora mapeando esses valores para númerico
Embarked ={"S":1,"C":2,"Q":3}
dataset_titanic['Embarked'] = dataset_titanic['Embarked'].map(Embarked)

In [ ]:
# Como está a questão dos nulos agora
(dataset_titanic.isnull().sum()).sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
Fare             0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
Survived         0
PassengerId      0
dtype: int64

In [ ]:
# Olhando novamente o dataframe
dataset_titanic.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
# Vamos mudar os dados categorigos de sexo para numerico
sex_mapa = {"male":0, "female":1}
dataset_titanic['Sex'] = dataset_titanic['Sex'].map(sex_mapa)
dataset_titanic.head(15)

In [ ]:
# Podemos pegar a feature nome e criar alguma feateure a partir do nome de tramento
# Criando uma nova feature a partir do titulo (nome de tratamento)
dataset_titanic ['Titulo'] = dataset_titanic['Name'].str.extract('([A-Za-z]+)\.',expand=False) 

pd.crosstab(dataset_titanic['Titulo'],dataset_titanic['Sex'])

# Abaixo o count por titulo com sexo

In [ ]:
 #Agrupando os titulos menos comum
dataset_titanic['Titulo'] = dataset_titanic['Titulo'].replace(['Lady','Capt','Col','Don','Dr','Major','Rev','Jonkheer','Dona'],'Rare') 
dataset_titanic['Titulo'] = dataset_titanic['Titulo'].replace(['Countess','Lady','Sir'],'Royal')
dataset_titanic['Titulo'] = dataset_titanic['Titulo'].replace('Mlle','Miss')
dataset_titanic['Titulo'] = dataset_titanic['Titulo'].replace('Ms','Miss')
dataset_titanic['Titulo'] = dataset_titanic['Titulo'].replace('Mme','Mrs')

dataset_titanic[['Titulo','Survived']].groupby(['Titulo'], as_index=False).mean()

# Agrupando os titulos forma de tratamento por % sobreviventes

Tranformando atributos agora para numerico

In [ ]:
# Transformar titulo em numerico
Titulo = {"Mr":1,"Miss":2, "Mrs":3, "Master":4, "Royal":5, "Rare":6}
dataset_titanic['Titulo']= dataset_titanic['Titulo'].map(Titulo)
dataset_titanic['Titulo'] = dataset_titanic['Titulo'].fillna(0)

In [ ]:
# transformar AgeGroup para numerico

AgeGroup = {"Unknow":0,"Baby":1, "Child":2, "Teenager":3, "Student":4, "Young Adult":5,"Adult":6, "Senior":7}
dataset_titanic['AgeGroup']= dataset_titanic['AgeGroup'].map(AgeGroup)

In [ ]:
# Como está a questão dos nulos agora
(dataset_titanic.isnull().sum()).sort_values(ascending=False)

In [ ]:
# Agora que foi criados as novas features, elimnando valores ausentes

dataset_titanic =  dataset_titanic.dropna(subset =['Age'])

In [ ]:
# Como está a questão dos nulos agora
(dataset_titanic.isnull().sum()).sort_values(ascending=False)

In [ ]:
# Eliminando features com pouco relevancia
dataset_titanic = dataset_titanic.drop(['Name', 'PassengerId'], axis =1)

# Abaixo ficamos apenas com númericos

In [ ]:
dataset_titanic['AgeGroup'] = dataset_titanic['AgeGroup'].astype('int64')
dataset_titanic['Age'] = dataset_titanic['Age'].astype('int64')
dataset_titanic['Fare'] = dataset_titanic['Fare'].astype('int64')

In [ ]:
dataset_titanic.dtypes

In [ ]:
dataset_titanic.head()
# Sobraram as seguintes features
# Survived = Variavel dependente
# PClass = Classe que o passageiro embarcou
# sex = 0 homem 1 mulher
# Idade = idade do passageiro
# SibSP = relação familiar
# Parch = valor pago no ticket
# Embarked = porto que foi embarcado
# AgeGroup = Faixa etária por grupo
# CabinBool = os atributos que existia numero da cabine foi defini 0 e 1 se continha ou não
# Titulo = forma de tratamento do passageiro extraido da coluna nome 


In [ ]:
# Analisando o dataframe
dataset_titanic.describe()

Aqui vamos rodar a regressão logistica !

In [ ]:
# Train test split considerando todas varivaveis

predictors = dataset_titanic.drop(["Survived"], axis =1)
target = dataset_titanic ["Survived"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size =0.2, random_state =0 )

In [ ]:

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val)* 100, 2)
print(acc_logreg)

In [ ]:
# Rodando o modelo removendo valor do ticket e tarifa

predictors = dataset_titanic.drop(["Survived","Parch", "Fare"], axis =1)
target = dataset_titanic ["Survived"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size =0.2, random_state =0 )



In [ ]:
# Regressão logistica
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val)* 100, 2)
print(acc_logreg)

Podemos ver acima que com a remoção de algumas features a acuracia do modelo subiu de 81,82 para 83,92.